In [1]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import wandb
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [3]:
trainfile = "tripled.train.10k.tsv"
devfile   = "mydevset_triples.tsv"

In [4]:
#train_df = pd.read_csv(trainfile, sep="\t", names=["text_a", "text_b", "labels"])
# Further scaling down the training set for reasonable sweep runtime
#train_df = train_df.head(10000)

In [5]:
#eval_df = pd.read_csv(devfile, sep="\t")[["q_text", "p_text", "relevance"]]
#eval_df.columns = ["text_a", "text_b", "labels"]

In [6]:
sweep_config = {
    "name": "ms_marco_sweep_distilroberta_base_full_smaller_triples_lazy",
    "method": "bayes",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "parameters": {
        "learning_rate": {"min": 0, "max": 4e-4},
        "num_train_epochs": {"min": 1, "max": 3}
    },
    "early_terminate": {"type": "hyperband", "min-iter": 6}
}

In [7]:
sweep_id = wandb.sweep(sweep_config, project="TREC-Covid")

Create sweep with ID: revtmojb
Sweep URL: https://app.wandb.ai/khituras/TREC-Covid/sweeps/revtmojb


In [8]:
model_args = ClassificationArgs()
model_args.lazy_loading = True
model_args.lazy_loading_start_line = 0
model_args.lazy_text_a_column = 0
model_args.lazy_text_b_column = 1
model_args.lazy_labels_column = 2
model_args.eval_batch_size = 16
model_args.evaluate_during_training = False
model_args.evaluate_during_training_silent = False
model_args.evaluate_during_training_steps = 1000
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.learning_rate = 4e-4
model_args.manual_seed = 4
model_args.max_seq_length = 256
model_args.multiprocessing_chunksize = 5000
model_args.no_cache = True
model_args.no_save = True
model_args.num_train_epochs = 3
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.train_batch_size = 16
model_args.gradient_accumulation_steps = 1
model_args.train_custom_parameter_only = False
model_args.n_gpu = 1
model_args.dataloader_num_workers = 1
model_args.labels_list = [0, 1]
model_args.wandb_project = "TREC-Covid"
model_args.early_stopping_metric = "accuracy"
model_args.early_stopping_metric_minimize = False

In [9]:
def train():
    wandb.init()
    model_args.wandb_kwargs = {"id": wandb.run.id}
    model = ClassificationModel(
        "roberta",
        "distilroberta-base",
        use_cuda=True,
        cuda_device=0,
        sweep_config=wandb.config,
        args=model_args
    )
    model.train_model(
        trainfile, eval_df=devfile, accuracy=lambda x,y: sklearn.metrics.accuracy_score(x,[round(p) for p in y])
    )
    res = model.eval_model(devfile, accuracy=lambda x,y: sklearn.metrics.accuracy_score(x,[round(p) for p in y]))
    wandb.log({"accuracy":res[0]["accuracy"]})
    
    wandb.join()

In [10]:
wandb.agent(sweep_id, train)

INFO:wandb.wandb_agent:Running runs: []
INFO:wandb.wandb_agent:Agent received command: run
INFO:wandb.wandb_agent:Agent starting run with config:
	learning_rate: 7.487312664066734e-05
	num_train_epochs: 1


wandb: Agent Starting Run: 41afdt7h with config:
	learning_rate: 7.487312664066734e-05
	num_train_epochs: 1
wandb: Agent Started Run: 41afdt7h


INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOjQxYWZkdDdoOlRSRUMtQ292aWQ6a2hpdHVyYXM=
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_143727-41afdt7h/config.yaml
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers
INFO:wandb.run_manager:file/dir created: /home/faessler/work/Research/trec-covid/wandb/run-20200730_143727-41afdt7h/wandb-metadata.json
INFO:wandb.run_manager:file/dir created: /home/faessler/work/Research/trec-covid/wandb/run-20200730_143727-41afdt7h/wandb-history.jsonl
INFO:wandb.run_manager:file/dir created: /home/faessler/work/Research/trec-covid/wandb/

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


INFO:wandb.run_manager:system metrics and metadata threads started
INFO:wandb.run_manager:checking resume status, waiting at most 10 seconds
INFO:wandb.run_manager:resuming run from id: UnVuOnYxOjQxYWZkdDdoOlRSRUMtQ292aWQ6a2hpdHVyYXM=
INFO:wandb.run_manager:upserting run before process can begin, waiting at most 10 seconds
INFO:wandb.run_manager:saving pip packages
INFO:wandb.run_manager:initializing streaming files api
INFO:wandb.run_manager:unblocking file change observer, beginning sync with W&B servers


...

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150915-2ygtf7aj/wandb-events.jsonl
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-history.jsonl
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150915-2ygtf7aj/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /h

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0


INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-metadata.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-summary.json
INFO:wandb.run_manager:file/dir modified: /home/faessler/work/Research/trec-covid/wandb/run-20200730_150924-2ygtf7aj/wandb-history.jsonl
wandb: Ctrl-c pressed. Waiting for runs to end. Press ctrl-c again to terminate them.


Process Process-12:
Exception in thread Thread-30:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/site-packages/tensorboardX/event_file_writer.py", line 202, in run
    data = self._queue.get(True, queue_wait_duration)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 108, in get
    res = self._recv_bytes()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 383, in _recv
    raise EOFError
EOFError

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99,